# Pràctica 2

L'objectiu d'aquesta segona pràctica es demostrar que heu assolit els conceptes que s'han explicar a l'assignatura i s'han practicat a les sessions presencials, relacionats amb el disseny i l'ús de xarxes neurals. 

Aquesta pràctica consta de 3 enunciats dels quals només heu de realitzar el que us hagi tocat per sorteig.

**Condicions**
1. El model que solucioni el problema estarà basat en xarxes neurals, aquestes s'han d'entrenar i avaluar emprant la llibreria _Pytorch_.
2. Es demana que com a mínim s'avaluin 2 models diferents: un que ha d'estar creat per vosaltres i un altre que es basi en una xarxa ja existent. Evidentment es permeten modificacions de la ja existen per adaptar-ho al problema que es vol resoldre.
3. El resultat del treball serà un informe on s'expliqui el procés que s'ha dut a terme per arribar a la que considereu que és millor solució. El document serà en format `pdf`. Podreu adjuntar una carpeta amb el codi i recursos que trobeu necessaris per comprovar la veracitat del que explicau al document.
4. Aquest document ha de tenir un llenguatge formal i tècnic i ha d'estar correctament estructurat:
    - Introducció al problema
    - Solucions considerades (dades, característiques, models, mètriques)
    - Experiments realitzats
    - Resultats dels experiments
    - Conclusions
5. A més del document explicatiu s'ha d'adjuntar un fitxer amb els pesos del millor entrenament de cada una de les xarxes que heu emprat (la que heu dissenyat vosaltres i la que ja existia), de tal manera que el professor pugui validar els resultats sense haver de repetir l'entrenament. Sense l'adjunció d'aquests fitxers la pràctica no es podrà aprovar.
6. Les dades depenen de cada un dels tres enunciats i les trobareu en el seu apartat.


**Avaluació**

- El treball es durà a terme en parelles.
- El professor es reserva la possibilitat de convocar als grups a una revisió de la pràctica de forma presencial.
- Només està permés emprar tècniques de disseny i entrenament vistes a classe.
- Tot el que no està fet pels alumnes ha d'estar referenciat, en cas contrari es considerarà com una còpia.


**Data d'entrega**

- Aquest treball s'entrega dia 15 de gener.
- Es realitzarà una tutoria dilluns 9 de gener a les 15:30.
---

## Enunciat 1: Classificació

El problema que heu de resoldre en aquest cas és un problema de classificació amb el conjunt de dades _Horses or Human_ dataset [enllaç](https://laurencemoroney.com/datasets.html). És un conjunt de dades generat per ordinador en el que trobareu dues classes diferents: persones i cavalls (500 imatges de cavalls i 547 imatges de persones). També dos subconjunts de dades ja definits: entrenament  i validació. Les imatges tenen una mida de 300x300 pixels i es troben en RGB.

A més de la feina de classificació i presentació dels resultats amb el conjunt de dades que es proporciona, també es demana que construiu un petit conjunt d'imatges (entre 10 i 20) de persones i cavalls reals com a conjunt de test i obtingueu les mesures rendiment adients per aquestes dades.


**Exemples del dataset**
<div style="display:flex">
     <div style="flex:1;padding-right:10px;">
          <img src="img/human01-16.png" width="200"/>
     </div>
     <div style="flex:1;padding-left:10px;">
          <img src="img/horse03-3.png" width="200"/>
     </div>
</div>

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import os

In [47]:
class CustomDataset(Dataset):

    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform
        self.all_files = [f for f in os.listdir(self.path) if f.endswith('.png')]
        self.labels = [f.split('.')[0] for f in self.all_files]
       
    def __len__(self):
        return len(self.all_files)
    
    def __getitem__(self, index):
        image = Image.open(os.path.join(self.path, self.all_files[index]))
        label = self.labels[index]

        if self.transform:
            image = self.transform(image)
        image = transforms.ToTensor()(image)
        return image, label


In [48]:
train_batch_size = 64
test_batch_size = 100

dataset = CustomDataset(path = "data/train/")
data_loader = DataLoader(dataset, batch_size=train_batch_size, shuffle=True)

for batch in data_loader:
    data, labels = batch
    print(labels)
# Creamos los datasets de entrenamiento y test, estan dentro de data/train
# y data/test


('horse-226', 'human-865', 'horse-120', 'horse-495', 'horse-379', 'horse-347', 'horse-322', 'horse-115', 'human-928', 'horse-293', 'human-982', 'human-817', 'horse-336', 'horse-163', 'horse-407', 'human-640', 'horse-75', 'horse-6', 'human-570', 'human-842', 'horse-106', 'human-826', 'human-622', 'human-769', 'human-971', 'horse-117', 'human-682', 'horse-426', 'horse-279', 'human-910', 'human-1004', 'horse-219', 'horse-16', 'human-905', 'horse-285', 'human-748', 'human-966', 'horse-216', 'human-800', 'human-993', 'horse-294', 'horse-351', 'horse-197', 'human-942', 'human-893', 'human-521', 'human-691', 'human-920', 'human-681', 'horse-334', 'human-958', 'human-651', 'human-959', 'human-535', 'human-809', 'human-658', 'horse-464', 'horse-496', 'human-790', 'human-874', 'horse-360', 'human-649', 'horse-479', 'horse-312')
('human-941', 'human-805', 'human-827', 'human-974', 'horse-443', 'human-808', 'horse-73', 'human-965', 'human-778', 'horse-138', 'human-944', 'horse-384', 'horse-21', 'h

('human-796', 'horse-419', 'horse-471', 'human-527', 'horse-49', 'horse-164', 'horse-185', 'human-578', 'human-576', 'horse-203', 'human-846', 'human-506', 'human-811', 'horse-40', 'human-539', 'horse-101', 'horse-424', 'horse-378', 'horse-195', 'horse-193', 'human-913', 'human-660', 'human-760', 'human-747', 'horse-429', 'horse-251', 'human-754', 'human-567', 'horse-470', 'horse-54', 'horse-386', 'human-909', 'horse-93', 'human-611', 'human-729', 'horse-337', 'horse-287', 'horse-271', 'human-565', 'human-655', 'human-762', 'human-519', 'human-614', 'human-612', 'horse-39', 'horse-275', 'horse-346', 'human-723', 'human-872', 'horse-425', 'horse-119', 'horse-198', 'horse-427', 'horse-490', 'horse-486', 'human-964', 'human-744', 'horse-339', 'horse-190', 'human-768', 'human-536', 'horse-135', 'horse-454', 'horse-179')
('human-574', 'human-916', 'human-995', 'human-932', 'horse-104', 'human-919', 'human-631', 'horse-317', 'horse-37', 'horse-461', 'human-1005', 'horse-290', 'human-739', 'h